In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

/Users/gastonrieder/.pyenv/versions/3.7.8/envs/general/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
ecobici_geojson = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/estaciones-bicicletas-publicas/nuevas-estaciones-bicicletas-publicas.geojson'
ecobici = gpd.read_file(ecobici_geojson)

In [4]:
comunas_geojson = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/comunas/CABA_comunas.geojson'
comunas = gpd.read_file(comunas_geojson)

In [5]:
comunas['COMUNAS'] = comunas['COMUNAS'].astype('int')
comunas.drop(columns=['AREA'], inplace=True)

In [6]:
comunas = comunas.to_crs(epsg=3857)
comunas['area'] = comunas.geometry.area / 10**6
comunas = comunas.to_crs(epsg=4326)

In [7]:
ecobici['nombre'] = ecobici['nombre'].str.extract('\d{3}\s.?([A-zÁ-ú0-9\s*]+)')
ecobici['nombre'] = ecobici['nombre'].str.upper()

In [8]:
mapa_caba = folium.Map((ecobici.geometry[0].y,ecobici.geometry[0].x), zoom_start = 12, min_zoom=11)

In [9]:
comunas_al_mapa = comunas['geometry'].apply(lambda x: folium.GeoJson(x).add_to(mapa_caba))

In [10]:
ecobici_cluster = MarkerCluster()

In [11]:
def puntos_ecobici(df):
    nombre_ecobici = df['nombre']
    popup = nombre_ecobici
    estacion_ecobici = folium.Marker((df['geometry'].y, df['geometry'].x), 
                                     popup=popup,
                                     icon=folium.Icon(color="green", icon="info-sign"))
    estacion_ecobici.add_to(ecobici_cluster)

In [12]:
estaciones_ecobici_al_mapa = ecobici.apply(lambda x: puntos_ecobici(x), axis=1)

In [13]:
ecobici_cluster.add_to(mapa_caba)

In [14]:
ecobici_en_comunas = gpd.sjoin(comunas, ecobici, op = 'contains')

In [15]:
ecobici_por_comuna = ecobici_en_comunas.groupby('COMUNAS').size()

In [16]:
ecobici_por_comuna_df = ecobici_por_comuna.to_frame(name='n_ecobici').reset_index()

In [17]:
ecobicis_en_comunas_df = pd.merge(comunas, ecobici_por_comuna_df, on='COMUNAS')

In [18]:
ecobicis_en_comunas_df['densidad_ecobici'] = ecobicis_en_comunas_df.apply(lambda x: x.n_ecobici/x.area, axis=1)

In [19]:
folium.Choropleth(geo_data=ecobici_en_comunas,
                    name='geometry',
                    data=ecobicis_en_comunas_df,
                    columns=['COMUNAS', 'densidad_ecobici'],
                    key_on='feature.properties.COMUNAS',
                    fill_color='YlGn',
                    fill_opacity=0.75,
                    line_opacity=0.5,
                    legend_name='densidad de estaciones de ecobici por comuna en CABA').add_to(mapa_caba)
folium.LayerControl().add_to(mapa_caba)

In [20]:
mapa_caba